# EDA de expresión diferencial (volcano, MA plot, DEGs)

Este notebook es un punto de partida para explorar tablas de expresión diferencial
(como la que has subido, p. ej. `PRJNA608927_Rapun_final_merged.tsv`).

Flujo básico:

1. Cargar la tabla (TSV/CSV/XLSX).
2. Detectar automáticamente los *contrastes* a partir de columnas del tipo `*_logFC`, `*_AveExpr`, `*_adj.P.Val`.
3. Elegir contraste y dibujar:
   - Volcano plot
   - MA plot
4. Sacar tabla de genes diferencialmente expresados (DEGs) con filtros de |logFC| y FDR.
5. Explorar un gen concreto a través de todos los contrastes.

Puedes adaptar este notebook y, más adelante, portar la lógica a una app de Streamlit.


In [ ]:
import re
from typing import Dict, Optional, Tuple

import numpy as np
import pandas as pd
import altair as alt

# Si usas Jupyter clásico, puede ayudarte este renderer:
alt.renderers.enable("default")

print("Versión pandas:", pd.__version__)

In [ ]:
def read_table(path: str, sep: Optional[str] = "auto", nrows_preview: int = 5) -> Tuple[pd.DataFrame, pd.DataFrame]:
    """Lee una tabla de expresión diferencial.
    
    - Si `sep='auto'` intenta detectar el separador con una lectura previa.
    - Devuelve (df, preview_df).
    """
    path = str(path)
    lower = path.lower()
    if lower.endswith((".xlsx", ".xls")):
        df = pd.read_excel(path)
    else:
        if sep is None or sep == "auto":
            # Sniff del separador en las primeras filas
            sniff = pd.read_csv(path, sep=None, engine="python", nrows=1000)
            sep_real = sniff.attrs.get("delimiter", ",")
            print(f"Separador detectado: '{sep_real}'")
            df = pd.read_csv(path, sep=sep_real)
        else:
            df = pd.read_csv(path, sep=sep)
    preview = df.head(nrows_preview).copy()
    return df, preview


def detectar_contrastes(df: pd.DataFrame) -> Dict[str, Dict[str, Optional[str]]]:
    """
    Detecta contrastes tipo:
      PREFIX_logFC, PREFIX_AveExpr, PREFIX_adj.P.Val, PREFIX_P.Value
    
    Devuelve un diccionario:
      prefix -> { 'logFC': col_name, 'AveExpr': col_name|None, 'adjP': col_name|None, 'P': col_name|None }
    """
    patrones = {
        "logFC": re.compile(r"(.+?)_logFC$", re.IGNORECASE),
        "AveExpr": re.compile(r"(.+?)_AveExpr$", re.IGNORECASE),
        "adjP": re.compile(r"(.+?)_adj\.P\.Val$", re.IGNORECASE),
        "P": re.compile(r"(.+?)_P\.Value$", re.IGNORECASE),
    }

    info: Dict[str, Dict[str, Optional[str]]] = {}

    for col in df.columns:
        col = str(col)
        for tipo, pat in patrones.items():
            m = pat.match(col)
            if m:
                pref = m.group(1)
                if pref not in info:
                    info[pref] = {"logFC": None, "AveExpr": None, "adjP": None, "P": None}
                if tipo == "logFC":
                    info[pref]["logFC"] = col
                elif tipo == "AveExpr":
                    info[pref]["AveExpr"] = col
                elif tipo == "adjP":
                    info[pref]["adjP"] = col
                elif tipo == "P":
                    info[pref]["P"] = col
                break

    # Filtrar solo contrastes con al menos logFC + (adjP o P)
    validos = {}
    for pref, d in info.items():
        if d["logFC"] is not None and (d["adjP"] is not None or d["P"] is not None):
            validos[pref] = d
    return validos


def elegir_columna_gen(df: pd.DataFrame) -> Optional[str]:
    """Intenta adivinar la columna que contiene el identificador del gen."""
    candidatos = ["SYMBOL", "gene_id", "Geneid", "GeneID", "gene", "Gene", "locus_tag"]
    for c in candidatos:
        if c in df.columns:
            return c
    return None


def pretty_contrast_name(pref: str) -> str:
    """Limpia un poco el nombre del contraste para visualización."""
    # Ejemplo: PRJNA608927_Rapun_wt-agrA -> wt-agrA
    s = re.sub(r"^PRJNA\d+_[^_]+_", "", pref)
    return s

In [ ]:
# ==== 1) CARGA DE TU TABLA ====
#
# Cambia 'ruta_al_fichero' por la ruta real de tu fichero.
# Por ejemplo, si estás en el mismo directorio que el .tsv:
#   path = "PRJNA608927_Rapun_final_merged.tsv"
#
# Si el archivo tiene tabuladores, deja sep='auto' o sep='\t'.

path = "PRJNA608927_Rapun_final_merged.tsv"  # <-- ajusta esto a tu ruta
df, preview = read_table(path, sep="auto", nrows_preview=5)

print(df.shape)
preview

In [ ]:
# ==== 2) DETECTAR CONTRASTES Y COLUMNA DE GEN ====

info_contrastes = detectar_contrastes(df)
print(f"Contrastes detectados: {len(info_contrastes)}")

for i, (pref, meta) in enumerate(info_contrastes.items(), start=1):
    print(f"{i:2d}. {pref}")
    print("    logFC  :", meta["logFC"])
    print("    AveExpr:", meta["AveExpr"])
    print("    adjP   :", meta["adjP"] or meta["P"])
    print()

gene_col = elegir_columna_gen(df)
if gene_col is None:
    print("⚠️ No se detectó automáticamente la columna de gen; elige una:")
    print(df.columns.tolist())
else:
    print("Columna de gen detectada:", gene_col)

In [ ]:
def preparar_deg_por_contraste(
    df: pd.DataFrame,
    gene_col: str,
    contraste: str,
    info_contrastes: Dict[str, Dict[str, Optional[str]]]
) -> pd.DataFrame:
    """Devuelve un DataFrame con columnas: gene, logFC, AveExpr, padj, neg_log10_padj."""
    meta = info_contrastes[contraste]
    logfc_col = meta["logFC"]
    ave_col = meta["AveExpr"]
    padj_col = meta["adjP"] if meta["adjP"] is not None else meta["P"]

    work = df.copy()

    rename_map = {
        gene_col: "gene",
        logfc_col: "logFC",
        padj_col: "padj",
    }
    if ave_col is not None:
        rename_map[ave_col] = "AveExpr"

    work = work.rename(columns=rename_map)

    if "AveExpr" not in work.columns:
        work["AveExpr"] = np.nan

    work["padj"] = pd.to_numeric(work["padj"], errors="coerce")
    work["logFC"] = pd.to_numeric(work["logFC"], errors="coerce")
    work["neg_log10_padj"] = -np.log10(np.clip(work["padj"].values, 1e-300, 1.0))

    return work[["gene", "logFC", "AveExpr", "padj", "neg_log10_padj"]]


# ==== 3) ELIGE UN CONTRASTE (MANUALMENTE) ====
# Escoge uno de los 'pref' listados arriba:
contraste = list(info_contrastes.keys())[0]  # por ejemplo, el primero
print('Usando contraste:', contraste, '→', pretty_contrast_name(contraste))

deg = preparar_deg_por_contraste(df, gene_col, contraste, info_contrastes)
deg.head()

In [ ]:
# ==== 4) VOLCANO Y MA PLOT ====
#
# Ajusta estos umbrales según lo necesites:
thr_logfc = 1.0   # |logFC| mínimo
thr_padj = 0.05   # FDR (adj.P.Val) máximo


def categorizar(row, thr_logfc=1.0, thr_padj=0.05):
    if np.isnan(row["logFC"]) or np.isnan(row["padj"]):
        return "No evaluable"
    if abs(row["logFC"]) >= thr_logfc and row["padj"] <= thr_padj:
        return "Up" if row["logFC"] > 0 else "Down"
    return "No sig"


deg["categoria"] = deg.apply(categorizar, axis=1, thr_logfc=thr_logfc, thr_padj=thr_padj)

# Volcano plot
volcano = alt.Chart(deg).mark_circle(size=40, opacity=0.7).encode(
    x=alt.X("logFC:Q", title="log2 Fold Change"),
    y=alt.Y("neg_log10_padj:Q", title="-log10(adj.P.Val)"),
    color=alt.Color("categoria:N", title="Categoría"),
    tooltip=["gene", "logFC", "padj"]
).properties(
    width=500,
    height=400,
    title=f"Volcano plot – {pretty_contrast_name(contraste)}"
)

volcano

In [ ]:
# MA plot
ma = alt.Chart(deg).mark_circle(size=40, opacity=0.7).encode(
    x=alt.X("AveExpr:Q", title="AveExpr (expresión media)"),
    y=alt.Y("logFC:Q", title="log2 Fold Change"),
    color=alt.Color("categoria:N", title="Categoría"),
    tooltip=["gene", "AveExpr", "logFC", "padj"]
).properties(
    width=500,
    height=400,
    title=f"MA plot – {pretty_contrast_name(contraste)}"
)

ma

In [ ]:
# ==== 5) TABLA DE DEGs ====
#
# Reutilizamos los mismos umbrales thr_logfc y thr_padj definidos arriba.

mask_deg = (deg["logFC"].abs() >= thr_logfc) & (deg["padj"] <= thr_padj)
deg_filt = deg.loc[mask_deg].copy().sort_values("padj")
print(f"Genes significativos: {deg_filt.shape[0]} de {deg.shape[0]}")
deg_filt.head(20)

In [ ]:
# ==== 6) EXPLORADOR PARA UN GEN CONCRETO A TRAVÉS DE TODOS LOS CONTRASTES ====
#
# Escribe el identificador del gen que quieres explorar:
gene_interes = "agrA"  # por ejemplo; ajusta a tu caso

# Filtramos la fila del gen de interés
sub = df[df[gene_col].astype(str) == gene_interes].copy()
print(f"Filas encontradas para {gene_interes}: {sub.shape[0]}")
sub.head(3)

In [ ]:
# Para ese gen, sacamos logFC / padj / AveExpr por contraste

filas = []
for pref, meta in info_contrastes.items():
    logfc_col = meta["logFC"]
    padj_col = meta["adjP"] if meta["adjP"] is not None else meta["P"]
    ave_col = meta["AveExpr"]

    if sub.empty:
        logfc_val = np.nan
        padj_val = np.nan
        ave_val = np.nan
    else:
        logfc_val = sub[logfc_col].iloc[0] if logfc_col in sub.columns else np.nan
        padj_val = sub[padj_col].iloc[0] if padj_col in sub.columns else np.nan
        ave_val = sub[ave_col].iloc[0] if ave_col and ave_col in sub.columns else np.nan

    filas.append({
        "Contraste": pretty_contrast_name(pref),
        "ID_interno": pref,
        "logFC": logfc_val,
        "AveExpr": ave_val,
        "adj.P.Val": padj_val
    })

resumen_gene = pd.DataFrame(filas).sort_values("Contraste")
resumen_gene

In [ ]:
# Pequeño plot de barras con logFC por contraste para ese gen
plot_logfc_gene = alt.Chart(resumen_gene).mark_bar().encode(
    x=alt.X("Contraste:N", sort="-y"),
    y=alt.Y("logFC:Q", title="log2 Fold Change"),
    tooltip=["Contraste", "logFC", "adj.P.Val"]
).properties(
    width=600,
    height=400,
    title=f"logFC por contraste para {gene_interes}"
)

plot_logfc_gene